# COGS 108 - Data Checkpoint

# Names

- Costin Smilovici
- George Gavrilov
- Karen Gong
- Aleene Baiza
- Vedan Desai

<a id='research_question'></a>
# Research Question

Does one US political party have a tendency to disproportionately use more negative sentiment on Twitter than the other and if so, what motivates this?

# Dataset(s)

Dataset 1
- Dataset Name: Tweets made by Representatives and Senators of the 115th and 116th Congress
- Link to the dataset: https://tweetsets.library.gwu.edu/datasets
- Number of observations: 3,674,077 with 4 datasets combined
- Description: 4 datasets ("congress116-senate-ids.txt", "congress116-house-ids.txt", "representatives-1.txt", "senators-1.txt") that we later combine into one in order to have a collection of all Twitter accounts that belong to Representatives and Senators of the 115th and 116th Congress. The dataset has one column, which is a column of all IDs of all those Twitter accounts. That will help us to identify if any tweets belong to any of the accounts within this dataset.

Dataset 2
- Dataset name: Twitter API "Tweet Lookup" Utility
Link to dataset: https://github.com/twitterdev/Twitter-API-v2-sample-code/blob/main/Tweet-Lookup/get_tweets_with_bearer_token.py
- Number of observations: The last week's worth of tweets for any regular user, and the entire tweet history for all - public figures. Thus in our case, exactly the number of observations we want: Approximately 40000.
- Description: This dataset has the ability to contain at least 19 different features, all of which are relatively useful for some purpose. However for our purposes we will be using the following:
    - id
        - A string data type that is the unique identifier of a tweet.
    - text
        - A string data type that is the actual UTF-8 text of the Tweet. See twitter-text for details on what characters are currently considered valid.
    - author_id
        - A string data type that is the unique identifier of the User who posted this Tweet.
    - created_at
        - A date (ISO 8601) data type that is the creation time of the Tweet.
    - entities (hashtags)
        - A json object that contains the start index of the hashtags in the tweet.
    - public_metrics
        - A json object that contains engagement metrics like retweet count, reply count, like count, and quote count.

Dataset 3
- Dataset Name: Politicians tracked by Politwoops 
- Link to Dataset: [Politicians tracked by Politwoops](https://www.propublica.org/datastore/dataset/politicians-tracked-by-politwoops) 
- Number of Observations: 2193 
- Description: This data set documents deleted tweets from campaign or official accounts of federal officeholders, candidates, governors, and gubernatorial candidates. The specific columns that we would use would just be twitter_id and party_name but all column descriptions are listed below:

    - user_name:
        - A string of the Twitter account's username
    - twitter_id: 
        - A unique numeric Twitter ID for the "user_name" account. 
    - party_name: 
        - A string of the Twitter user's political affiliation. 
    - state
        - A string of the Twitter user's state that they represent or represented. 
    - office_title
        - The Twitter users' office title.
    - account_type: 
        - A string describing what type of account this Twitter user is using. 
    - first_name
        - First name of the politician
    - middle_name
        - Middle name of the politician
    - last_name
        - Last name of the politician
    - suffix
        - Suffix on the politician within Congress
    - status
        - 1 if the Twitter account is active and 0 if it is not
    - collecting
        - Boolean data type that seems to display whether or not Politwoops is collecting from this acct.
    - showing
        - Boolean data type that seems to display whether or not Politwoops is showing from this acct.
    - bioguide_id
        - A string data type that serves as a unique identifier
    - gender
        - A string data type that serves as an identifier of a person's gender
    - fec_candidate_id
        - A string data type that represents an acct's Federal Election Commission candidate ID
    - district
        - A numerical data type that represents the congressional district of an account
    - branch
        - A string data type that represents the branch of government a political figure serves in 
        
All three of these datasets are used to create our finalized dataset, although not all features make it all the way through to the end. The first dataset listed above that contains tweet ids for all congress members of the 115th and 116th sessions, so that is used to query the Twitter api and generate the second of our datasets above which will also contain user id. After that, we join this dataset to the 3rd dataset to add in a feature that represents the political affiliation of each observation, this yields our final dataset.

# Setup

In [1]:
import pandas as pd
import numpy as np
import os
from requests_oauthlib import OAuth1Session
import json

In [2]:
# API_KEY_SEC = "fbrAQcqbIERo3YZ90Zr49Ohmj8dfl1F9oaLPLnrMutV6xC5p7t"
# API_KEY = "MlxRUs0anXY8ix9mkzmIkgndW"
# Contains the API key information that will be needed for querying the Twitter API
API_KEY_SEC = "7OhJs7m9jxh7LSoEKiGxPvew0saGE92kexzEOqLhjn8Jjuw85j"
API_KEY = "X0d8SI0eYV8FP4Csz0EPU2So7"
# The sample size for the number of tweets we will use
SAMPLE_SIZE = 100000

# Data Cleaning

To obtain the clean data we need to merge the existing datasets together as they are given in a tidy form already.

First we can load the data containing all tweets from all congress members from the 115th and 116th congresses

In [3]:
senate_tweets = pd.read_csv("./DataFiles/congress116/congress116-senate-ids.txt",header=None)
house_tweets = pd.read_csv("./DataFiles/congress116/congress116-house-ids.txt",header=None)
house_tweets2 = pd.read_csv("./DataFiles/congress115/representatives-1.txt",header=None)
senate_tweets2 = pd.read_csv("./DataFiles/congress115/senators-1.txt",header=None)

We can then concatenate the data into a single column and drop duplicates such the result is one column that contains all of the unique tweets from congress members from the 115th and 116th congresses

In [4]:
poli_tweets = pd.concat([senate_tweets,house_tweets,senate_tweets2,house_tweets2])
poli_tweets = poli_tweets.drop_duplicates()
poli_tweets.head()

,0
0,1136262855588810753
1,1134178369673863169
2,1133015265619402754
3,1136007350706475016
4,1136003439270477824


In [5]:
poli_tweets.shape

(3674077, 1)

Because there are 3,674,077 tweets in the final column, we will only take a sample from this of which we defined the sample size as 100,000. Recall from our Project Proposal that we only need a sample of 39,000 meaning that 100,000 should be sufficient.

In [7]:
np.random.seed(57)
sample = poli_tweets.sample(SAMPLE_SIZE,replace=False)
sample.head()

,0
190865,775729553738260484
300498,941068661351157761
77400,1058425426668933120
139908,1063257348351832065
678898,1140972496503607297


In [8]:
sample.shape

(100000, 1)

The Twitter API only allows for searching 100 tweets at a time in the form of a comma seperated string. Because of this, the following loop turns our sample into a list containing 1000 strings of 100 comma seperated IDs.

In [10]:
search_arr = []
for i in range(SAMPLE_SIZE//100):
    search_str = ""
    tweet = sample.iloc[(i*100):(i*100 + 100),0]
    for ele in tweet:
        search_str += str(ele) + ","
    search_str = search_str[:-1]
    search_arr += [search_str]

This code is Twitter's authentication code for python so it can largely be ignored

**Source: https://github.com/twitterdev/Twitter-API-v2-sample-code/blob/main/Tweet-Lookup/get_tweets_with_user_context.py**

In [8]:
# To set your enviornment variables in your terminal run the following line:
consumer_key = API_KEY
consumer_secret = API_KEY_SEC

request_token_url = "https://api.twitter.com/oauth/request_token"
oauth = OAuth1Session(consumer_key, client_secret=consumer_secret)

try:
    fetch_response = oauth.fetch_request_token(request_token_url)
except ValueError:
    print(
        "There may have been an issue with the consumer_key or consumer_secret you entered."
    )

resource_owner_key = fetch_response.get("oauth_token")
resource_owner_secret = fetch_response.get("oauth_token_secret")
print("Got OAuth token: %s" % resource_owner_key)

# Get authorization
base_authorization_url = "https://api.twitter.com/oauth/authorize"
authorization_url = oauth.authorization_url(base_authorization_url)
print("Please go here and authorize: %s" % authorization_url)
verifier = input("Paste the PIN here: ")

# Get the access token
access_token_url = "https://api.twitter.com/oauth/access_token"
oauth = OAuth1Session(
    consumer_key,
    client_secret=consumer_secret,
    resource_owner_key=resource_owner_key,
    resource_owner_secret=resource_owner_secret,
    verifier=verifier,
)
oauth_tokens = oauth.fetch_access_token(access_token_url)


access_token = oauth_tokens["oauth_token"]
access_token_secret = oauth_tokens["oauth_token_secret"]

# Make the request
oauth = OAuth1Session(
    consumer_key,
    client_secret=consumer_secret,
    resource_owner_key=access_token,
    resource_owner_secret=access_token_secret,
)

Got OAuth token: Fofg-gAAAAABcUJ0AAABgLTzIFY
Please go here and authorize: https://api.twitter.com/oauth/authorize?oauth_token=Fofg-gAAAAABcUJ0AAABgLTzIFY
Paste the PIN here: 4569815


This code calls the API for each of the 100 tweet long strings in our list containing all of these strings. This then creates an array of dataframes that we can concatenate into one large dataframe. Note not all tweets are able to be obtained likely because of changed accessing rights on the tweet itself. Nonetheless, most of the tweets should be available given that these are policitian twitter accounts.

In [ ]:
API_Tweets = []

for i in range(SAMPLE_SIZE//100):
    params = {"ids": search_arr[i], "tweet.fields": "created_at,author_id,entities,public_metrics"}
    response = oauth.get(
        "https://api.twitter.com/2/tweets", params=params
    )
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(response.status_code, response.text)
        )
    json_response = response.json()
    dataframed_response = pd.DataFrame(json_response["data"])[["entities","public_metrics","author_id","text","created_at"]]
    API_Tweets += [dataframed_response]

Now that we have the 1000 tweet dataframes in an array we can do pd.concat to get the entire dataframe containing tweets from politicians.

In [10]:
tweet_text = pd.concat(API_Tweets)
tweet_text.head()

,entities,public_metrics,author_id,text,created_at
0,"{'annotations': [{'start': 0, 'end': 6, 'proba...","{'retweet_count': 4, 'reply_count': 0, 'like_c...",431105217,Garrett was 1 of just a handful of fringe extr...,2016-09-13T16:15:12.000Z
1,"{'annotations': [{'start': 0, 'end': 10, 'prob...","{'retweet_count': 93, 'reply_count': 15, 'like...",250188760,Republicans have reportedly settled on even BI...,2017-12-13T22:13:47.000Z
2,"{'annotations': [{'start': 32, 'end': 46, 'pro...","{'retweet_count': 3247, 'reply_count': 468, 'l...",17494010,What the @AARP has to say about President Trum...,2018-11-02T18:27:40.000Z
3,"{'hashtags': [{'start': 52, 'end': 71, 'tag': ...","{'retweet_count': 424, 'reply_count': 358, 'li...",30354991,This incredibly powerful video is a must watch...,2018-11-16T02:28:00.000Z
4,"{'annotations': [{'start': 6, 'end': 12, 'prob...","{'retweet_count': 5, 'reply_count': 12, 'like_...",2964949642,"Spoke America’s global leadership, diplomacy &...",2019-06-18T13:20:16.000Z


We can take a quick look at the dtypes to see if there is any cleaning that is needed:

In [12]:
tweet_text

,entities,public_metrics,author_id,text,created_at
0,"{'annotations': [{'start': 0, 'end': 6, 'proba...","{'retweet_count': 4, 'reply_count': 0, 'like_c...",431105217,Garrett was 1 of just a handful of fringe extr...,2016-09-13T16:15:12.000Z
1,"{'annotations': [{'start': 0, 'end': 10, 'prob...","{'retweet_count': 93, 'reply_count': 15, 'like...",250188760,Republicans have reportedly settled on even BI...,2017-12-13T22:13:47.000Z
2,"{'annotations': [{'start': 32, 'end': 46, 'pro...","{'retweet_count': 3247, 'reply_count': 468, 'l...",17494010,What the @AARP has to say about President Trum...,2018-11-02T18:27:40.000Z
3,"{'hashtags': [{'start': 52, 'end': 71, 'tag': ...","{'retweet_count': 424, 'reply_count': 358, 'li...",30354991,This incredibly powerful video is a must watch...,2018-11-16T02:28:00.000Z
4,"{'annotations': [{'start': 6, 'end': 12, 'prob...","{'retweet_count': 5, 'reply_count': 12, 'like_...",2964949642,"Spoke America’s global leadership, diplomacy &...",2019-06-18T13:20:16.000Z
...,...,...,...,...,...
92,"{'hashtags': [{'start': 55, 'end': 67, 'tag': ...","{'retweet_count': 80, 'reply_count': 0, 'like_...",105290638,RT @COParksWildlife: Share this if you love Co...,2017-05-20T18:36:37.000Z
93,"{'hashtags': [{'start': 54, 'end': 70, 'tag': ...","{'retweet_count': 49, 'reply_count': 0, 'like_...",991721030631780354,RT @Essence: We are emboldened by the changema...,2018-10-27T13:59:53.000Z
94,"{'urls': [{'start': 72, 'end': 95, 'url': 'htt...","{'retweet_count': 0, 'reply_count': 1, 'like_c...",950328072,@LukeBeige Thank you for your comment. Here i...,2016-09-23T01:56:13.000Z
95,"{'urls': [{'start': 99, 'end': 121, 'url': 'ht...","{'retweet_count': 10, 'reply_count': 29, 'like...",19739126,Our country must take a new path with new poli...,2015-10-03T19:18:49.000Z


Notice that they are all objects and from looking at the data frame we have some dictionaries. Thus we need to do some cleaning as follows:
- **entities:** Extract the hashtags that are contained in the tweets for further analysis.
- **public_metrics:** Extract all of the values and make them their own columns for engagement analysis.
- **author_id:** Turn the column into a numerical column to allow for future merging
- **created_at:** Turn the column into a datetime column to allow time based analysis

In [13]:
# Concat all of the returned tweet dataframes 
tweet_text = pd.concat(API_Tweets)
# Author IDs needs to be numeric for merging purposes
tweet_text["author_id"] = tweet_text["author_id"].astype(int)

# Get the hashtags from the entities dictionaries and convert it into a list of hashtage
tweet_text["hashtags"] = (tweet_text["entities"]
                          .apply(lambda x: [ele['tag'] for ele in x.get("hashtags",[])] if type(x) == dict else []))

# Get all of the value names from the public metrics dictionary inorder to make the columns
public_metrics_cols = list(tweet_text["public_metrics"].iloc[0].keys())
public_metrics_cols

# Create the columns containing the public metrics data
for col in public_metrics_cols:
    tweet_text[col] = (tweet_text["public_metrics"]
                           .apply(lambda x: x.get(col,0)))
    
# Convert the created_at value to datetime
tweet_text["created_at"] = pd.to_datetime(tweet_text["created_at"])

# Drop the columns with dictionaries
tweet_text = tweet_text.drop(columns=["entities","public_metrics"])

tweet_text

,author_id,text,created_at,hashtags,retweet_count,reply_count,like_count,quote_count
0,431105217,Garrett was 1 of just a handful of fringe extr...,2016-09-13 16:15:12+00:00,[VAWA],4,0,5,0
1,250188760,Republicans have reportedly settled on even BI...,2017-12-13 22:13:47+00:00,[],93,15,97,6
2,17494010,What the @AARP has to say about President Trum...,2018-11-02 18:27:40+00:00,[],3247,468,4817,165
3,30354991,This incredibly powerful video is a must watch...,2018-11-16 02:28:00+00:00,[TransAwarenessWeek],424,358,1173,45
4,2964949642,"Spoke America’s global leadership, diplomacy &...",2019-06-18 13:20:16+00:00,[],5,12,29,2
...,...,...,...,...,...,...,...,...
92,105290638,RT @COParksWildlife: Share this if you love Co...,2017-05-20 18:36:37+00:00,"[publiclands, COPublicLandsDay]",80,0,0,0
93,991721030631780354,RT @Essence: We are emboldened by the changema...,2018-10-27 13:59:53+00:00,[TheChisholmList],49,0,0,0
94,950328072,@LukeBeige Thank you for your comment. Here i...,2016-09-23 01:56:13+00:00,[],0,1,0,0
95,19739126,Our country must take a new path with new poli...,2015-10-03 19:18:49+00:00,[],10,29,4,0


With the author's account ids and the tweet text, we can now use the "Politicians tracked by Politwoops" dataset from ProPublica Data Store to get the party affiliation of the representatives.

In [14]:
account_party = pd.read_csv("./DataFiles/political_party.csv")
account_party.head()

,user_name,twitter_id,party_name,state,office_title,account_type,first_name,middle_name,last_name,suffix,status,collecting,showing,bioguide_id,gender,fec_candidate_id,district,branch
0,MorgunSholtyNV,1491675900257058818,Republican,NV,House Candidate,campaign,Morgun,NaN,Sholty,NaN,1,True,True,NaN,M,H2NV01280,1.0,H
1,BrogForNevada,1491469539313397762,Republican,NV,House Candidate,campaign,David,NaN,Brog,NaN,1,True,True,NaN,M,H2NV01298,1.0,H
2,CandiceforMN_D1,1503790339404058627,Democrat,MN,House Candidate,campaign,Candice,Deal,Bartell,NaN,1,True,True,NaN,F,H2MN01256,1.0,H
3,KenNavitsky,2800279758,Republican,MN,House Candidate,campaign,Ken,NaN,Navitsky,NaN,1,True,True,NaN,M,H2MN01298,1.0,H
4,NelsTPierson,499072725,Republican,MN,House Candidate,campaign,Nels,NaN,Pierson,NaN,1,True,True,NaN,M,H2MN01306,1.0,H


We can finally finish off by doing a merge and removing the duplicates from the final dataframe to get the cleaned dataset.

Note that there are no NaN values in the merged DataFrame. Because of that and because we modified the values where needed, we can assume that this data is in a clean form

In [15]:
cleaned_data = (tweet_text
                    .merge(account_party,left_on="author_id",right_on="twitter_id")
                    [["text","party_name","created_at","hashtags","retweet_count","reply_count","like_count","quote_count"]])
non_duplicate_idx = cleaned_data["text"].drop_duplicates().index
cleaned_data = cleaned_data.iloc[non_duplicate_idx].reset_index().drop(columns=["index"])
cleaned_data["text"] = cleaned_data["text"].str.replace("\r"," ")
cleaned_data.head()

False


,text,party_name,created_at,hashtags,retweet_count,reply_count,like_count,quote_count
0,Garrett was 1 of just a handful of fringe extr...,Democrat,2016-09-13 16:15:12+00:00,[VAWA],4,0,5,0
1,"""No one fights harder than Josh to bring back ...",Democrat,2018-06-19 14:41:25+00:00,[],0,1,2,0
2,"""Two years in Washington is not much time, yet...",Democrat,2018-11-01 23:20:35+00:00,[],7,0,20,0
3,TODAY'S THE DAY! Stopped by the Hackensack Tra...,Democrat,2018-11-06 16:20:00+00:00,[ElectionDay],0,2,11,0
4,Bergen Congressman Reacts To 'Horrific' Shooti...,Democrat,2017-06-15 19:26:50+00:00,[],1,0,3,0
